### Converting an Allen-Style Taxonomy Spreadsheet to CAS Format  

This notebook demonstrates how to convert an Allen-style taxonomy spreadsheet into the CAS (Cell Annotation Schema) format. The process includes the following steps:  

1. **Generate CAS**: Convert the spreadsheet data into the CAS format.
2. **Validate CAS against h5ad**: Compare CAS and AnnData to check if the annotations and the hierarchy are the same.
3. **Populate IDs**: Add corresponding Cell IDs to the CAS from a related Anndata (h5ad) file.  
4. **Merge Data**: Integrate the updated CAS into the `var` field of the Anndata object.  

This workflow uses a subset of the WMBO dataset. Example Allen-style spreadsheet is derived from the `CB Glut (CS20230722_CLAS_29)` subset of the Mouse Cell-type annotations (https://www.nature.com/articles/s41586-023-06812-z#Sec49, Table 7)

This notebook demonstrates how to carry out these steps in Python, but most can also be done via the [cas-tools command-line interface](https://cellannotation.github.io/cas-tools/cli.html).

### How to Run the Notebooks

For detailed instructions on setting up and running the notebooks, please refer to the [README.md](https://github.com/cellannotation/cas-tools/blob/main/notebooks/README.md) in the notebooks directory.


#### Example of an Allen-Style Spreadsheet  

To keep things simple, we use the `CB Glut (CS20230722_CLAS_29)` subset from the Mouse Cell-Type Annotations dataset (refer to [Nature article, Supplementary Table 7](https://www.nature.com/articles/s41586-023-06812-z#Sec49)).

In [2]:
import pandas as pd

pd.read_csv("./data/wmb_class_29_annotation.tsv", delimiter='\t')[:4]

,cluster_id,cluster,supertype,subclass,class,neighborhood,anatomical_annotation,notes,CCF_broad.freq,CCF_acronym.freq,...,CTX.subclass_id,CTX.subclass_id.1,CTX.neighborhood_id,CTX.neighborhood_label,CTX.size,taxonomy_id,cell_set_accession.cluster,cell_set_accession.supertype,cell_set_accession.subclass,cell_set_accession.class
0,5197,5197 CB Granule Glut_1,1154 CB Granule Glut_1,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,DCO VCO,NaN,"MY:0.35,CB:0.33,NA:0.26","DCO:0.19,FL:0.11,arb:0.09,VCO:0.09,DN:0.08,PFL...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5197,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29
1,5198,5198 CB Granule Glut_1,1154 CB Granule Glut_1,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,DCO VCO,NaN,"CB:0.56,MY:0.24,NA:0.19","FL:0.25,PFL:0.17,DCO:0.13,VCO:0.09,arb:0.08,NA...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5198,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29
2,5199,5199 CB Granule Glut_1,1154 CB Granule Glut_1,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,DCO VCO,NaN,"MY:0.4,CB:0.38,NA:0.18","DCO:0.29,DN:0.12,arb:0.09,FL:0.09,PFL:0.06,VCO...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5199,CS20230722_SUPT_1154,CS20230722_SUBC_314,CS20230722_CLAS_29
3,5200,5200 CB Granule Glut_2,1155 CB Granule Glut_2,314 CB Granule Glut,29 CB Glut,NN-IMN-GC,NOD PFL,NaN,"CB:0.77,NA:0.22","PFL:0.17,NOD:0.17,FL:0.13,NA:0.11,arb:0.1,UVU:...",...,NaN,NaN,NaN,NaN,NaN,CCN202307220,CS20230722_CLUS_5200,CS20230722_SUPT_1155,CS20230722_SUBC_314,CS20230722_CLAS_29


#### Import Spreadsheet into CAS  

Import assumes that the source spreadsheet will have:

* One row per cluster (where cluster is the most granular cell set)
* A column containing cluster_ids
* A spreadsheet representation of hierarchy with columns for each leve in the hierarchy.
* A set of columns specific to the sheet (author categories)

To import an Allen-style spreadsheet into the CAS, we need a simple mapping file. This includes some general metadata and specification of heirarchy fields.  Rank refers to the level of a heirarchy field - with 0 being the most granular. The mapping file must specify cluster_id and cluster_name require specific field types.  All other hierarchy fields are typed as cell set.  Here is a configuration for our example sheet:

In [29]:
!less ./data/wmb_ingestion_config.yaml

taxonomy_id: CS20230722
brain_region_names:
  - Whole Brain
species_names:
  - Mouse
author_name: Hongkui Zeng
title: Whole Mouse Brain taxonomy

fields:
  - column_name: cluster_id
    column_type: cluster_id
    rank: 0

  - column_name: cluster
    column_type: cluster_name
    rank: 0

  - column_name: supertype
    column_type: cell_set
    rank: 1

  - column_name: subclass
    column_type: cell_set
./data/wmb_ingestion_config.yamltaxonomy_id: CS20230722
brain_region_names:
  - Whole Brain
species_names:
  - Mouse
author_name: Hongkui Zeng
title: Whole Mouse Brain taxonomy

fields:
  - column_name: cluster_id
    column_type: cluster_id
    rank: 0

  - column_name: cluster
    column_type: cluster_name
    rank: 0

  - column_name: supertype
    column_type: cell_set
    rank: 1

  - column_name: subclass
    column_type: cell_set
    rank: 2
:

In [ ]:
# Import to CAS

import json
from cas.ingest.ingest_user_table import ingest_data, ingest_user_data

cas_data = ingest_user_data("./data/wmb_class_29_annotation.tsv", "./data/wmb_ingestion_config.yaml", True)
wmb_class_29_cas = cas_data.as_dictionary()
print(json.dumps(wmb_class_29_cas, indent=2)[:2000])


In [ ]:
# Or we can use reporting tools from cas-tools to view dataframes:

from cas import reports
reports.get_all_annotations(wmb_class_29_cas)

#### Retrieve AnnData File for `CB Glut`

Assumed starting point - an AnnData file with annotations corresponding to the hierarchy in the spreadsheet.

Download the AnnData file corresponding to `CB Glut (CS20230722_CLAS_29)`. The original [WMB-10Xv2](https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#expression_matrices/WMB-10Xv2/20230630/) and [WMB-10Xv3](https://allen-brain-cell-atlas.s3.us-west-2.amazonaws.com/index.html#expression_matrices/WMB-10Xv3/20230630/) AnnData files were generated based on dissection.  

These files were merged and then split into 34 top-level classes. The resulting AnnData files for each class can be accessed at the following links:  
- [Class 01](http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_01.h5ad)  
- [Class 02](http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_02.h5ad)  
- ...  
- [Class 34](http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_34.h5ad)  

The file specific to `CB Glut` is included in this collection.

In [7]:
!wget -N http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_29.h5ad

--2025-02-11 16:21:30--  http://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_29.h5ad
Resolving cellular-semantics.cog.sanger.ac.uk (cellular-semantics.cog.sanger.ac.uk)... 193.62.203.62, 193.62.203.61, 193.62.203.63
Connecting to cellular-semantics.cog.sanger.ac.uk (cellular-semantics.cog.sanger.ac.uk)|193.62.203.62|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_29.h5ad [following]
--2025-02-11 16:21:30--  https://cellular-semantics.cog.sanger.ac.uk/public/merged_CS20230722_CLAS_29.h5ad
Connecting to cellular-semantics.cog.sanger.ac.uk (cellular-semantics.cog.sanger.ac.uk)|193.62.203.62|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘merged_CS20230722_CLAS_29.h5ad’ not modified on server. Omitting download.



In [8]:
from cas.file_utils import read_anndata_file

anndata = read_anndata_file("./merged_CS20230722_CLAS_29.h5ad")
anndata.obs[:3]

,cell_barcode,library_label,tissue,tissue_ontology_term_id,neurotransmitter,class,subclass,supertype,cluster,organism,disease,assay
cell_label,,,,,,,,,,,,
AAACCCAAGAACAAGG-472_A05,AAACCCAAGAACAAGG,L8TX_201217_01_G07,Cerebellum,UBERON:0002037,Glut,29 CB Glut,314 CB Granule Glut,1155 CB Granule Glut_2,5201 CB Granule Glut_2,Mus musculus,normal,10x 3' v2
AAACCCAAGAATCCCT-473_A06,AAACCCAAGAATCCCT,L8TX_201217_01_A08,Cerebellum,UBERON:0002037,Glut,29 CB Glut,314 CB Granule Glut,1155 CB Granule Glut_2,5201 CB Granule Glut_2,Mus musculus,normal,10x 3' v3
AAACCCAAGACTACCT-225_A01,AAACCCAAGACTACCT,L8TX_200227_01_F10,Medulla,UBERON:0001896,Glut,29 CB Glut,314 CB Granule Glut,1154 CB Granule Glut_1,5197 CB Granule Glut_1,Mus musculus,normal,10x 3' v2


#### Analyse CAS and AnnData


In [ ]:
# Manual version to be replaced by CAS built in when ready
obs = anndata.obs 

labelsets = [labelset.name for labelset in sorted(cas_data.labelsets, key=lambda x: x.rank)]
print(labelsets)

# assert cluster counts are the same
cas_clusters = { annotation.cell_label for annotation in cas_data.annotations if annotation.labelset == "cluster"}
anndata_clusters = set(obs["cluster"].unique())
assert cas_clusters == anndata_clusters

# assert hierarchies are the same
for annotation in cas_data.annotations:
    if labelsets.index(annotation.labelset) < len(labelsets) -1:
        # list all parent names for the given annotation in the AnnData
        parent_labelset = labelsets[labelsets.index(annotation.labelset) + 1]
        parent_names = obs[obs[annotation.labelset] == annotation.cell_label][parent_labelset].unique()
        # assert has only one parent
        assert len(parent_names) == 1
        # assert AnnData parent is same with CAS parent 
        assert annotation.parent_cell_set_name == parent_names[0]

#### Populate CAS with Cell IDs  

Add Cell IDs to the CAS using data from the AnnData file.  

This first checks that the heirarchy of nested cell sets in the AnnData file matches the heirarchy in the CAS file (as derived from the Allen spreadsheet). If two cell set are identical but one has a higher rank then the other, this is also treated as valid heirarchy.

If the heirarchies match, Cell_IDs are added to the CAS representation.

By default, cell IDs are only added at the most granular level (clusters), as Cell IDs for other levels in the hierarchy can be derived.

In [10]:
add_cell_ids?

Signature:
add_cell_ids(
    cas: dict,
    ad: Union[pandas.core.frame.DataFrame, cap_anndata.backed_df.CapAnnDataDF],
    labelsets: list = None,
)
Docstring:
Add/update CellIDs to CAS from matching AnnData file.

Parameters:
    cas: CAS JSON object
    ad: anndata object
    labelsets: List of labelsets to update with IDs from AnnData. If value is null, rank '0' labelset is used. The
    labelsets should be provided in order, starting from rank 0 (leaf nodes) and ascending to higher ranks.
File:      ~/Documents/GitHub/cas-tools/venv/lib/python3.9/site-packages/cas/populate_cell_ids.py
Type:      function

In [9]:
from cas.populate_cell_ids import add_cell_ids

labelset_names = [labelset.name for labelset in sorted(cas_data.labelsets, key=lambda x: x.rank)]
cas = add_cell_ids(wmb_class_29_cas, anndata, labelset_names)

print(json.dumps(cas["annotations"][5], indent=2)[:1500])

AttributeError: 'AnnData' object has no attribute 'columns'

#### Merge CAS to Anndata

Adds CAS to uns of the AnnData file.  Note that further checks are carried out here to assess whether cell set membership matches between the CAS file and the AnnData file.  In this case they do match, but this also protects against files being out of sync.

In [ ]:
from cas.anndata_conversion import merge_cas_object
import anndata as ad

anndata.file.close()
merge_cas_object(wmb_class_29_cas, "./merged_CS20230722_CLAS_29.h5ad", True, "./merged_CS20230722_CLAS_29_v2.h5ad")

In [ ]:
from cas.file_utils import read_cas_from_anndata

cas_read = read_cas_from_anndata("./merged_CS20230722_CLAS_29_v2.h5ad")
cas_read.get_all_annotations().head(5)

In [26]:
anndata.file.close()

In [27]:
import cas.file_utils

In [28]:
cas.file_utils.write_json_file?

Signature:
cas.file_utils.write_json_file(
    cas: cas.model.CellTypeAnnotation,
    out_file: str,
    print_undefined: bool = False,
)
Docstring:
Writes cell type annotation object to a json file.
:param cas: cell type annotation object to serialize.
:param out_file: output file path.
:param print_undefined: prints null values to the output json if true. Omits undefined values from the json output if
File:      ~/Documents/GitHub/cas-tools/venv/lib/python3.9/site-packages/cas/file_utils.py
Type:      function